# Searchlight analysis

NOW LIVE ON HOFFMAN: SEE run_searchlight.py

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

if sys.platform == 'darwin':
    cfg = os.path.join("/Users", "njchiang", "CloudStation", "Grad",
                       "Research", "montilab-ucla", "analogy", "config", "project.json")
    plat = "osx"

elif sys.platform == "linux":
    import platform
    if platform.linux_distribution()[0] == "debian":
        cfg = os.path.join("/home", "njchiang", "data", "CloudStation", "Grad",
                           "Research", "montilab-ucla", "analogy", "config", "project.json")
        plat = "linux"
    else:
        cfg = os.path.join("/u", "project", "monti", "Analysis", "Analogy",
                           "code", "analogy", "config", "project.json")
        plat = "hoff"
else:
    cfg = os.path.join("D:\\", "CloudStation", "Grad",
                       "Research", "montilab-ucla", "analogy", "config", "project.json")
    plat = "win"

with open(cfg, "r") as f:
    projectSettings = json.load(f)

paths = projectSettings["filepaths"]["{}Paths".format(plat)]
sys.path.append(paths["github"])
sys.path.append(paths["code"])

In [3]:
from fmri.analogy_utils import analysisSettings, contrastSettings, order, \
    pu, pa, pv, compile_models, rsa, save_rois, load_rois, load_betas


Loading JSON config from D:\CloudStation\Grad\Research\montilab-ucla\analogy\config\analyses.json
Loading JSON config from D:\CloudStation\Grad\Research\montilab-ucla\analogy\config\contrasts.json
Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\trialorder_rsa_absorted.csv


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

In [5]:
# analysisSettings["searchlight"]["n_jobs"] = 6

In [6]:
maskname = "grayMatter-bin_mask"

for sub in projectSettings["subjects"].keys():
    
    mask = pu.load_img(paths["root"], "derivatives", sub, "masks", "{}.nii.gz".format(maskname))

    fmri_data, labels, bg_image = load_betas(projectSettings, sub, t="tstat-LSS", center=False, scale=False)
    selector = labels[labels.CD == 1]
    
    fmri_data = pu.index_img(fmri_data, selector.index)


#     op = StandardScaler()
#     scaledData = pa.op_by_label(fmri_data, labels['chunks'], op)
    # analysis
#     clf = SVC()
    clf = Pipeline(steps=[# ("variance_threshold", VarianceThreshold()),
                           ("scaling", StandardScaler()),
    #                        ("feature_select", SelectPercentile(f_classif, percentile=20)),
    #                        ("feature_select", SelectKBest(f_classif, k=100)),
                           ("svm", LinearSVC(C=0.05))
    #                        ("plr", LogisticRegression(C=0.05, penalty="l1", tol=0.01))
    ])
    cv = LeaveOneGroupOut()
    result = pa.searchlight(fmri_data, 
                            selector.Match, m=mask, cv=cv, 
                            groups=selector['chunks'],
                            **analysisSettings['searchlight'])
    
    pu.data_to_img(results.scores_, bg_image).to_filename(
        os.path.join(paths["root"], "analysis", sub, 
                     "multivariate", "searchlight", "lss", 
                     "{}_{}_CD_Match_searchlight.nii.gz".format(sub, maskname)))


Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\grayMatter-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-01_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-02_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-03_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-04_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-05_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-06_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-07_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: D:\fmri\Analogy\derivatives\sub-01\reg\BOLD_tem

KeyboardInterrupt: 

In [ ]:
def run_subject(sub, modelname, maskname="grayMatter-bin_mask", logger=None):

    fmri_data, labels, bg_image, mask = load_data_lss(sub, maskname,
                                                      t=contrastSettings[modelname]["t"],
                                                      normalize=False, logger=logger)

    conditionSelector = np.where(labels[contrastSettings[modelname]["label"]] != "None")

    op = StandardScaler()
    scaledData = pa.op_by_label(fmri_data, labels['chunks'], op, logger=logger)

    # analysis
    # output order: CD2AB, AB2CD
    analysisSettings["searchlight"]["estimator"] = SVC()
    cv = LeaveOneGroupOut()

    result = pa.searchlight(pu.unmask_img(scaledData[conditionSelector], mask),
                            labels.iloc[conditionSelector][contrastSettings[modelname]["label"]],
                            m=mask, cv=cv,
                            groups=labels.iloc[conditionSelector][contrastSettings[modelname]["grouping"]],
                            logger=logger,
                            **analysisSettings['searchlight'])

    outpath = os.path.join(paths["root"], "analysis", sub, "multviariate", "searchlight",
                           "{}-{}-cvsl.nii.gz".format(sub, modelname))
    pu.data_to_img(result.scores_, bg_image, logger=logger).to_filename(outpath)
    return result

OLD BELOW

Searchlight Analysis: Cross-validated AB

In [3]:
sublist = [i for i in projectSettings["subjects"].keys()]
# for i in range(15):
#     del(sublist[0])
# sublist

In [3]:
# for each subject
# mask = lIFGoperc_bin-mask
import numpy as np
import pandas as pd
analysisSettings["searchlight"]["n_jobs"] = 4
def runsubject(sub, mask, model, t="tstat"):
    labels=[]
    imgs=[]
    mask = pu.load_img(paths['root'], 
                      'derivatives', sub, 'masks', 
                      mask + '.nii.gz')
    for r in range(8):
        imgFile = os.path.join(paths['root'], 'derivatives', sub, 'betas', 
                               "{}_task-analogy_run-0{}_betas-{}-LSS.nii.gz".format(sub, r+1, t))
        labels.append(pu.load_labels(paths["root"],
                           'derivatives', sub, 'func', 
                            "{}_task-analogy_run-0{}_events.tsv".format(sub, r+1),
                           sep='\t'))
        imgs.append(pu.mask_img(imgFile, mask))
        labels[r]["chunks"] = pd.Series([r+1 for i in range(len(labels[r]))])
    #         resids.append(pa.mask_img(residFile, mask))
    #         run_data = noise_normalize_beta()
    labels = pd.concat(labels).reset_index()
    conditionSelector = np.where(labels['AB'] == 1)
    fmri_data = np.vstack(imgs)

    bg_image = pu.load_img(os.path.join(paths['root'], 
                                            'derivatives', 
                                            sub, 'reg', 'BOLD_template.nii.gz'))
#     fmri_data, these_labels, bg_image, mask = 

    # preprocessing
    from sklearn.preprocessing import StandardScaler
    op = StandardScaler()
    scaledData = pa.op_by_label(fmri_data, labels['chunks'], op)
    # analysis
    from sklearn.svm import SVC
    clf = SVC()
    from sklearn.model_selection import LeaveOneGroupOut
    cv = LeaveOneGroupOut()
    result = pa.searchlight(pu.unmask_img(scaledData[conditionSelector], mask), 
                            labels.iloc[conditionSelector][model], m=mask, cv=cv, 
                            groups=labels.iloc[conditionSelector]['chunks'],
                            **analysisSettings['searchlight'])

    outpath = os.path.join(paths["root"], "derivatives", sub, "searchlight",
                           "{}-{}-cvsl.nii.gz".format(sub, model))
    pu.data_to_img(result.scores_, bg_image).to_filename(outpath)
    return result

SyntaxError: invalid syntax (<ipython-input-3-4899e335f49a>, line 30)

In [7]:
[runsubject(s, 'graymatter-bin_mask', "ABMainRel", t="tstat") for s in sublist]

Reading file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-14/masks/graymatter-bin_mask.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-14/func/sub-14_task-analogy_run-01_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-14/betas/sub-14_task-analogy_run-01_betas-tstat-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-14/func/sub-14_task-analogy_run-02_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-14/betas/sub-14_task-analogy_run-02_betas-tstat-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-14/func/sub-14_task-analogy_run-03_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-14/betas/sub-14_task-analogy_run-03_betas-tstat-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-14/func/sub-14_task-analogy_run-04_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-14/betas/su

[SearchLight(cv=LeaveOneGroupOut(), estimator='svc',
       mask_img=<nibabel.nifti1.Nifti1Image object at 0x7f1faa39d438>,
       n_jobs=4, process_mask_img=None, radius=5, scoring=None, verbose=0),
 SearchLight(cv=LeaveOneGroupOut(), estimator='svc',
       mask_img=<nibabel.nifti1.Nifti1Image object at 0x7f1ffd98f278>,
       n_jobs=4, process_mask_img=None, radius=5, scoring=None, verbose=0),
 SearchLight(cv=LeaveOneGroupOut(), estimator='svc',
       mask_img=<nibabel.nifti1.Nifti1Image object at 0x7f2001ad6630>,
       n_jobs=4, process_mask_img=None, radius=5, scoring=None, verbose=0)]

In [4]:
%%bash
cd ~/data/fmri/Analogy/analysis
chance=0.50  # 0.34
betatype=lss
for m in ab2cd-cvsl # ABMainRel-cvsl
do
  echo ${m}
  for s in 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
  do
    echo sub-${s}
    flirt -in sub-${s}/multivariate/searchlight/sub-${s}-${m}.nii.gz -out group/multivariate/searchlight/${betatype}/sub-${s}-${m}.nii.gz -ref ../derivatives/standard/MNI152_T1_2mm_brain.nii.gz -applyxfm -init ../derivatives/sub-${s}/reg/BOLD_template_to_standard.mat
    fslmaths group/multivariate/searchlight/${betatype}/sub-${s}-${m} -bin -mul ${chance} group/multivariate/searchlight/${betatype}/chance
    fslmaths group/multivariate/searchlight/${betatype}/sub-${s}-${m} -sub group/multivariate/searchlight/${betatype}/chance group/multivariate/searchlight/${betatype}/sub-${s}-${m}
    rm group/multivariate/searchlight/${betatype}/chance.nii.gz
  done
  fslmerge -t group/multivariate/searchlight/${betatype}/group-${m}.nii.gz group/multivariate/searchlight/${betatype}/sub*-${m}.nii.gz
#   fslmaths analysis/multivariate/searchlight/lss/group-${m}.nii.gz -sub 0.3334 analysis/multivariate/searchlight/lss/group-${m}.nii.gz
  rm group/multivariate/searchlight/${betatype}/sub*-${m}.nii.gz
done

ABMainRel-cvsl
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16


In [5]:
%%bash
cd ~/data/fmri/Analogy/
for m in ABMainRel-cvsl
do
#   randomise -i analysis/encoding/group-${m}-searchlight.nii.gz -o analysis/encoding/n1000_ba10_sl_${m} -1 -n 1000 -T -x -v 5 -m standard/masks/aal-LOFC_mask.nii.gz --uncorrp
#   randomise -i analysis/encoding/group-${m}-searchlight.nii.gz -o analysis/encoding/n1000_svm_sl_${m} -1 -n 1000 -T -v 5 -x -m standard/masks/svm_abmainrel.nii.gz --uncorrp
  randomise -i analysis/group/multivariate/searchlight/lss/group-${m}.nii.gz -o analysis/group/multivariate/searchlight/lss/n1000_graymatter_${m} -1 -n 1000 -T -x -v 5 -m derivatives/standard/masks/grayMatter.nii.gz --uncorrp
done

randomise options: -i analysis/group/multivariate/searchlight/lss/group-ABMainRel-cvsl.nii.gz -o analysis/group/multivariate/searchlight/lss/n1000_graymatter_ABMainRel-cvsl -1 -n 1000 -T -x -v 5 -m derivatives/standard/masks/grayMatter.nii.gz --uncorrp 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
65536 sign-flips required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting permutation 22
Starting permutation 23
Start

In [19]:
%%bash
cd ~/data/fmri/Analogy/derivatives
for m in ABMainRel-cvsl
do
  fdr -i analysis/multivariate/searchlight/lss/n1000_graymatter_${m}_vox_p_tstat1 --oneminusp -m standard/masks/grayMatter.nii.gz -q 0.05 -a analysis/multivariate/searchlight/lss/n1000_graymatter_${m}_vox_fdrp_tstat1
#   fdr -i analysis/encoding/n1000_graymatter_sl_${m}_vox_p_tstat1 --oneminusp -m standard/masks/aal-LOFC_mask.nii.gz -q 0.05 -a analysis/encoding/n1000_ba10_sl_${m}_vox_fdrp_tstat1
#   fdr -i analysis/encoding/n1000_svm_sl_${m}_vox_p_tstat1 --oneminusp -m standard/masks/svm_abmainrel.nii.gz -q 0.05 -a analysis/encoding/n1000_svm_sl_${m}_vox_fdrp_tstat1
  fdr -i analysis/multivariate/searchlight/lss/n1000_graymatter_${m}_tfce_p_tstat1 --oneminusp -m standard/masks/grayMatter.nii.gz -q 0.05 -a analysis/multivariate/searchlight/lss/n1000_graymatter_${m}_tfce_fdrp_tstat1
#   fdr -i analysis/encoding/n1000_graymatter_sl_${m}_tfce_p_tstat1 --oneminusp -m standard/masks/aal-LOFC_mask.nii.gz -q 0.05 -a analysis/encoding/n1000_ba10_sl_${m}_tfce_fdrp_tstat1
#   fdr -i analysis/encoding/n1000_svm_sl_${m}_tfce_p_tstat1 --oneminusp -m standard/masks/svm_abmainrel.nii.gz -q 0.05 -a analysis/encoding/n1000_svm_sl_${m}_tfce_fdrp_tstat1

done



Probability Threshold is: 
0.00599998
Probability Threshold is: 
0.02


Searchlight AB 2 CD

In [4]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut

# for each subject
# mask = lIFGoperc_bin-mask
analysisSettings["searchlight"]["n_jobs"] = 2
def runsubject(sub, maskname, model="Tag", t="cope", normalize=False):
    mask = pu.load_img(paths['root'],
                       'derivatives', sub, 'masks',
                       maskname + '.nii.gz')

    labels = []
    imgs = []
    # whiten the data
    for r in range(8):
        imgFile = os.path.join(paths['root'], 'derivatives', sub, 'betas',
                               "{}_task-analogy_run-0{}_betas-{}-LSS.nii.gz".format(sub, r+1, t))
        labels.append(pu.load_labels(paths["root"],
                                     'derivatives', sub, 'func',
                                     "{}_task-analogy_run-0{}_events.tsv".format(sub, r+1),
                                     sep='\t'))
        labels[r]["chunks"] = pd.Series([r+1 for i in range(len(labels[r]))])
        residFile = os.path.join(paths['root'], 'derivatives', sub, 'func',
                               pu.format_bids_name(sub, 'task-analogy',
                                                   "run-0{}".format(r+1),
                                                   'resids-lss.nii.gz'))
        if normalize:
            maskedImg = pa.mask_img(imgFile, mask)
            resids = pa.mask_img(residFile, mask)
            imgs.append(rsa.noise_normalize_beta(maskedImg, resids))
        else:
            imgs.append(pa.mask_img(imgFile, mask))

    #         resids.append(pa.mask_img(residFile, mask))
    #         run_data = noise_normalize_beta()
    labels = pd.concat(labels).reset_index()
    conditionSelector = np.where(labels['Probe'] == 0)
    fmri_data = np.vstack(imgs)
    these_labels = labels.iloc[conditionSelector].reset_index()
    fmri_data = fmri_data[conditionSelector]
    bg_image = pu.load_img(os.path.join(paths['root'],
                                        'derivatives',
                                        sub, 'reg', 'BOLD_template.nii.gz'))
    y = []
    for _, r in these_labels.iterrows():
        if r["ABTag"] == "None":
            y.append(r["CDMainRel"])
        else:
            y.append(r["ABMainRel"])
            
    y = np.array(y)
    # preprocessing
    op = StandardScaler()
    scaledData = pa.op_by_label(fmri_data, these_labels['chunks'], op)
    

    # analysis
    # output order: CD2AB, AB2CD
    clf = SVC()
    cv = LeaveOneGroupOut()
    
    # automatically averages... crap
    
    result = pa.searchlight(pu.unmask_img(scaledData, mask), 
                            y, m=mask, cv=cv, 
                            groups=these_labels['TrialType'],
                            **analysisSettings['searchlight'])

    outpath = os.path.join(paths["root"], "derivatives", sub, "searchlight",
                           "{}-{}-cvsl.nii.gz".format(sub, model))
    pu.data_to_img(result.scores_, bg_image).to_filename(outpath)
    return result

In [5]:
del sublist[0:4]
sublist

['sub-05',
 'sub-06',
 'sub-07',
 'sub-08',
 'sub-09',
 'sub-10',
 'sub-11',
 'sub-12',
 'sub-13',
 'sub-14',
 'sub-15',
 'sub-16']

In [6]:
[runsubject(s, 'graymatter-bin_mask', "Tag", t="cope") for s in sublist]

Reading file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-05/masks/graymatter-bin_mask.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-05/func/sub-05_task-analogy_run-01_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-05/betas/sub-05_task-analogy_run-01_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-05/func/sub-05_task-analogy_run-02_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-05/betas/sub-05_task-analogy_run-02_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-05/func/sub-05_task-analogy_run-03_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-05/betas/sub-05_task-analogy_run-03_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-05/func/sub-05_task-analogy_run-04_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-05/betas/sub-0

OSError: [Errno 22] Invalid argument

In [17]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
def run_subject(sub, modelname, maskname="grayMatter-bin_mask"):

    fmri_data, labels, bg_image, mask = load_data_lss(sub, maskname,
                                                      t=contrastSettings[modelname]["t"],
                                                      normalize=False)

    conditionSelector = np.where(labels[contrastSettings[modelname]["label"]] != "None")

    op = StandardScaler()
    scaledData = pa.op_by_label(fmri_data, labels['chunks'], op)

    # analysis
    # output order: CD2AB, AB2CD
    analysisSettings["searchlight"]["estimator"] = SVC()
    cv = LeaveOneGroupOut()

    result = pa.searchlight(pu.unmask_img(scaledData[conditionSelector], mask),
                            labels.iloc[conditionSelector][contrastSettings[modelname]["label"]],
                            m=mask, cv=cv,
                            groups=labels.iloc[conditionSelector][contrastSettings[modelname]["grouping"]],
                            **analysisSettings['searchlight'])

    outpath = os.path.join(paths["root"], "analysis", sub, "multviariate", "searchlight",
                           "{}-{}-cvsl.nii.gz".format(sub, modelname))
    pu.data_to_img(result.scores_, bg_image).to_filename(outpath)
    return result

In [18]:
contrastSettings

{'ab2cd': {'t': 'cope', 'grouping': 'TrialType', 'label': 'MainRel'},
 'ABMainRel': {'t': 'cope', 'grouping': 'chunks', 'label': 'ABMainRel'},
 'CDMainRel': {'t': 'cope', 'grouping': 'chunks', 'label': 'ABMainRel'}}

In [20]:
run_subject("sub-16", modelname="ab2cd", maskname="grayMatter-bin_mask")

Reading file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/masks/grayMatter-bin_mask.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-01_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-16_task-analogy_run-01_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-02_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-16_task-analogy_run-02_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-03_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-16_task-analogy_run-03_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-04_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-1

OSError: [Errno 22] Invalid argument

In [9]:
fmri_data, labels, bg_image, mask = load_data_lss("sub-16", "grayMatter-bin_mask", t="cope", normalize=False)

Reading file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/masks/grayMatter-bin_mask.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-01_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-16_task-analogy_run-01_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-02_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-16_task-analogy_run-02_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-03_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-16_task-analogy_run-03_betas-cope-LSS.nii.gz
Loading label file from: /home/njchiang/data/fmri/Analogy/derivatives/sub-16/func/sub-16_task-analogy_run-04_events.tsv
Masking /home/njchiang/data/fmri/Analogy/derivatives/sub-16/betas/sub-1

In [16]:
labels.iloc[conditionSelector][contrastSettings["ab2cd"]["label"]]

0      3
1      1
3      3
4      2
6      1
7      1
9      1
10     1
12     1
13     3
15     1
16     1
18     2
19     1
21     3
22     3
24     3
25     3
27     2
28     1
30     2
31     1
33     3
34     2
36     1
37     3
39     3
40     3
42     2
43     2
      ..
819    3
820    3
822    3
823    3
825    2
826    2
828    3
829    3
831    3
832    2
834    2
835    2
837    3
838    2
840    3
841    3
843    3
844    1
846    1
847    1
849    1
850    1
852    1
853    3
855    1
856    2
858    2
859    3
861    2
862    2
Name: MainRel, Length: 576, dtype: object

In [14]:
labels.iloc[conditionSelector]

,index,Onset,Duration,TrialTag,ABTag,CDTag,Match,ABMainRel,CDMainRel,ABSubRel,...,ProbeCorr,ProbeArr,AB,CD,Probe,Intensity,TrialType,chunks,MainRel,SubRel
0,0,3.0195,2.0077,work:earn::stairs:ladder,work:earn,None,None,3,None,9,...,0,0,1,0,0,1.0,AB,1,3,9
1,1,6.5401,2.0037,work:earn::stairs:ladder,None,stairs:ladder,0,None,1,None,...,0,0,0,1,0,1.0,CD,1,1,2
3,3,11.5343,2.0094,heater:warmth::climb:descend,heater:warmth,None,None,3,None,7,...,0,0,1,0,0,1.0,AB,1,3,7
4,4,14.3997,2.0106,heater:warmth::climb:descend,None,climb:descend,0,None,2,None,...,0,0,0,1,0,1.0,CD,1,2,5
6,6,18.8014,2.0088,pretty:beautiful::teach:instruct,pretty:beautiful,None,None,1,None,1,...,0,0,1,0,0,1.0,AB,1,1,1
7,7,21.6623,2.0144,pretty:beautiful::teach:instruct,None,teach:instruct,1,None,1,None,...,0,0,0,1,0,1.0,CD,1,1,1
9,9,27.0008,2.0092,discount:price::heat:temperature,discount:price,None,None,1,None,3,...,0,0,1,0,0,1.0,AB,1,1,3
10,10,30.0711,2.0055,discount:price::heat:temperature,None,heat:temperature,1,None,1,None,...,0,0,0,1,0,1.0,CD,1,1,3
12,12,34.2579,2.0187,hill:mountain::danger:flee,hill:mountain,None,None,1,None,2,...,0,0,1,0,0,1.0,AB,1,1,2
13,13,37.2015,2.0082,hill:mountain::danger:flee,None,danger:flee,0,None,3,None,...,0,0,0,1,0,1.0,CD,1,3,8


In [21]:
None in [None, 1, 2]

True